In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark.sql(f"CREATE SCHEMA IF NOT EXISTS medalhao_case.download")
spark.sql(f"DROP VOLUME IF EXISTS medalhao_case.download.allData")
spark.sql(f"CREATE VOLUME IF NOT EXISTS medalhao_case.download.allData")
spark = SparkSession.builder.appName("CaseV-Credit").getOrCreate()


In [0]:


for schema in ['bronze','silver','gold']:
    # Lista de tabelas
    tables = spark.catalog.listTables(f'medalhao_case.{schema}')
    for t in tables:
        
        table_name = t.name
        temp_dir_path = f'/Volumes/medalhao_case/download/allData/{table_name}'

        # Escreve os dados em um volume
        spark.table(f'medalhao_case.{schema}.{table_name}').coalesce(1).write.format('csv').option("header", "true").mode("overwrite").save(temp_dir_path)

        # Busca o path
        files = dbutils.fs.ls(temp_dir_path)
        
        # Busca o arquivo part
        data_file = [f.path for f in files if f.name.startswith("part-") and f.name.endswith(".csv")][0]
        
        # Copia/renomeia o arquivo para o volume
        final_file_path = f'/Volumes/medalhao_case/download/allData/{table_name}.csv'
        dbutils.fs.cp(data_file, final_file_path)
        dbutils.fs.rm(temp_dir_path, True)

        print(f"Tabela {table_name} salva em: {final_file_path}")
    